# Feature engineering and Data Transformation

In [27]:

import json
import pandas as pd
import numpy as np

import os
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns


## Load Cleaned Data

In [28]:
data_train = pd.read_csv('../data/train_clean.csv')
data_test = pd.read_csv('../data/test_clean.csv')

In [29]:
data_train.head()

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,1026827,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-99.00,-99.00,0.0,No,No,No,Yes,No,No
1,1043384,2.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.99,0.99,0.0,No,No,No,Yes,No,No
2,1043696,2.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-99.00,-99.00,0.0,Yes,No,No,Yes,No,No
3,1043852,7.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.10,0.13,0.0,No,No,No,Yes,No,No
4,1044048,8.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-99.00,-99.00,0.0,Yes,No,No,Yes,No,No


## Handle Negative Inventory:

Finding: In EDA, describe() output showed national_inv has a minimum value of -27256.0.

Action: capping at 0 is often simpler and reasonable for inventory

In [30]:

data_train['national_inv'] = data_train['national_inv'].clip(lower=0)
data_test['national_inv'] = data_test['national_inv'].clip(lower=0)

In [31]:
data_train['national_inv'].describe()

count    1.687860e+06
mean     4.964937e+02
std      2.961518e+04
min      0.000000e+00
25%      4.000000e+00
50%      1.500000e+01
75%      8.000000e+01
max      1.233440e+07
Name: national_inv, dtype: float64

## Log Transformation for Skewed Features:

Finding: EDA plots and describe() showed heavy right-skewness and large ranges for features like national_inv, in_transit_qty, forecasts (forecast_3_month, etc.), sales (sales_1_month, etc.), and min_bank. 

Action: Apply a log transformation (e.g., np.log1p which handles zeros by calculating log(1+x)) to these highly skewed numerical features. Apply this to both train and test sets.

In [32]:
cols_to_transform = ['national_inv', 'in_transit_qty',
                     'forecast_3_month', 'forecast_6_month', 
                     'forecast_9_month', 'sales_1_month', 
                     'sales_3_month', 'sales_6_month', 
                     'sales_9_month', 'min_bank']

# Apply log1p transformation to handle zeros
# log1p(x) = log(1 + x)
data_train[cols_to_transform] = np.log1p(data_train[cols_to_transform])
data_test[cols_to_transform] = np.log1p(data_test[cols_to_transform])

In [33]:
# Function to calculate skewness stats
def print_skewness_comparison(df, columns):
    original_skew = df[columns].apply(lambda x: x.replace([-np.inf, np.inf], np.nan).skew())
    
    # Calculate log1p transformed skewness
    transformed_data = np.log1p(df[columns])
    transformed_skew = transformed_data.apply(lambda x: x.replace([-np.inf, np.inf], np.nan).skew())
    
    comparison = pd.DataFrame({
        'Original Skewness': original_skew,
        'Transformed Skewness': transformed_skew
    })
    
    comparison['Improvement'] = comparison['Original Skewness'].abs() - comparison['Transformed Skewness'].abs()
    comparison['Improvement %'] = (comparison['Improvement'] / comparison['Original Skewness'].abs()) * 100
    
    return comparison.round(3)

# Calculate and display skewness comparison
skewness_stats = print_skewness_comparison(data_train, cols_to_transform)
print("Skewness Analysis (closer to 0 is better):")
print("\nPositive skewness indicates right-tailed distribution")
print("Negative skewness indicates left-tailed distribution")
print("\nResults:")
print(skewness_stats)

# Additional summary
print("\nSummary:")
improved_features = skewness_stats[skewness_stats['Improvement'] > 0]
print(f"- {len(improved_features)} features showed reduced skewness after transformation")
print(f"- Average improvement: {improved_features['Improvement %'].mean():.1f}%")

Skewness Analysis (closer to 0 is better):

Positive skewness indicates right-tailed distribution
Negative skewness indicates left-tailed distribution

Results:
                  Original Skewness  Transformed Skewness  Improvement  \
national_inv                  0.711                -0.606        0.105   
in_transit_qty                2.625                 1.903        0.722   
forecast_3_month              1.837                 1.215        0.622   
forecast_6_month              1.527                 0.930        0.597   
forecast_9_month              1.381                 0.793        0.587   
sales_1_month                 1.763                 0.914        0.849   
sales_3_month                 1.293                 0.471        0.822   
sales_6_month                 1.058                 0.236        0.822   
sales_9_month                 0.942                 0.110        0.833   
min_bank                      1.688                 0.901        0.787   

                  Improv

## Handle Coded Numerical Values (Performance Averages)

Finding: EDA plots and conclusion highlighted that perf_6_month_avg and perf_12_month_avg have spikes at -99 and potentially 0, suggesting coded meanings (like 'no data' or 'perfect performance') rather than continuous values.

Action: Treat these as categorical-like features:

Binning: Create categories (e.g., 'No Data' for -99, 'Perfect' for 0, 'Below Average', 'Average', 'Above Average' for other ranges).

In [34]:
# First, let's examine the unique values in performance columns
print("Unique values in performance columns:")
print("\nperf_6_month_avg:", sorted(data_train['perf_6_month_avg'].unique()))
print("\nperf_12_month_avg:", sorted(data_train['perf_12_month_avg'].unique()))

# Function to categorize performance
def categorize_performance(value):
    if value == -99:
        return 'No Data'
    elif value == 0:
        return 'Perfect'
    elif value < 0.3:  #this  can adjust these thresholds based on domain knowledge
        return 'Above Average'
    elif value < 0.7:
        return 'Average'
    else:
        return 'Below Average'

# Create new categorical columns
data_train['perf_6_month_cat'] = data_train['perf_6_month_avg'].apply(categorize_performance)
data_train['perf_12_month_cat'] = data_train['perf_12_month_avg'].apply(categorize_performance)
data_test['perf_6_month_cat'] = data_test['perf_6_month_avg'].apply(categorize_performance)
data_test['perf_12_month_cat'] = data_test['perf_12_month_avg'].apply(categorize_performance)

# Look at the distribution of categories
print("\nDistribution of 6-month performance categories:")
print(data_train['perf_6_month_cat'].value_counts())
print("\nDistribution of 12-month performance categories:")
print(data_train['perf_12_month_cat'].value_counts())

Unique values in performance columns:

perf_6_month_avg: [np.float64(-99.0), np.float64(0.0), np.float64(0.01), np.float64(0.02), np.float64(0.03), np.float64(0.04), np.float64(0.05), np.float64(0.06), np.float64(0.07), np.float64(0.08), np.float64(0.09), np.float64(0.1), np.float64(0.11), np.float64(0.12), np.float64(0.13), np.float64(0.14), np.float64(0.15), np.float64(0.16), np.float64(0.17), np.float64(0.18), np.float64(0.19), np.float64(0.2), np.float64(0.21), np.float64(0.22), np.float64(0.23), np.float64(0.24), np.float64(0.25), np.float64(0.26), np.float64(0.27), np.float64(0.28), np.float64(0.29), np.float64(0.3), np.float64(0.31), np.float64(0.32), np.float64(0.33), np.float64(0.34), np.float64(0.35), np.float64(0.36), np.float64(0.37), np.float64(0.38), np.float64(0.39), np.float64(0.4), np.float64(0.41), np.float64(0.42), np.float64(0.43), np.float64(0.44), np.float64(0.45), np.float64(0.46), np.float64(0.47), np.float64(0.48), np.float64(0.49), np.float64(0.5), np.float64(

## Encode Categorical Features

Finding: While EDA focused on numerical, dataset description mentions several Yes/No columns (potential_issue, deck_risk, oe_constraint, ppap_risk, stop_auto_buy, rev_stop) which act as binary categories.

Action: Convert these Yes/No columns into numerical representations (0 and 1). (pd.get_dummies with drop_first=True or simple mapping like {'Yes': 1, 'No': 0}).

In [35]:
data_train.columns

Index(['sku', 'national_inv', 'lead_time', 'in_transit_qty',
       'forecast_3_month', 'forecast_6_month', 'forecast_9_month',
       'sales_1_month', 'sales_3_month', 'sales_6_month', 'sales_9_month',
       'min_bank', 'potential_issue', 'pieces_past_due', 'perf_6_month_avg',
       'perf_12_month_avg', 'local_bo_qty', 'deck_risk', 'oe_constraint',
       'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder',
       'perf_6_month_cat', 'perf_12_month_cat'],
      dtype='object')

In [36]:
# Let's also look at other potential categorical columns
binary_cols = ['potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 
               'stop_auto_buy', 'rev_stop', 'went_on_backorder']

print("\nUnique values in other potential categorical columns:")
for col in binary_cols:
    print(f"\n{col}:", sorted(data_train[col].unique()))


Unique values in other potential categorical columns:

potential_issue: ['No', 'Yes']

deck_risk: ['No', 'Yes']

oe_constraint: ['No', 'Yes']

ppap_risk: ['No', 'Yes']

stop_auto_buy: ['No', 'Yes']

oe_constraint: ['No', 'Yes']

ppap_risk: ['No', 'Yes']

stop_auto_buy: ['No', 'Yes']

rev_stop: ['No', 'Yes']

went_on_backorder: ['No', 'Yes']

rev_stop: ['No', 'Yes']

went_on_backorder: ['No', 'Yes']


In [37]:
# Drop the original performance columns since we now have categorical versions
columns_to_drop = ['perf_6_month_avg', 'perf_12_month_avg']
data_train.drop(columns=columns_to_drop, inplace=True)
data_test.drop(columns=columns_to_drop, inplace=True)

# Convert all binary columns to numeric (0/1) for machine learning
binary_cols = ['potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 
               'stop_auto_buy', 'rev_stop', 'went_on_backorder']

for col in binary_cols:
    data_train[col] = (data_train[col] == 'Yes').astype(int)
    data_test[col] = (data_test[col] == 'Yes').astype(int)

# Create dummy variables for the performance categories
# This will one-hot encode our new categorical columns
perf_columns = ['perf_6_month_cat', 'perf_12_month_cat']
data_train = pd.get_dummies(data_train, columns=perf_columns, prefix=perf_columns)
data_test = pd.get_dummies(data_test, columns=perf_columns, prefix=perf_columns)

# Verify the transformations
print("Binary columns after encoding (sample):")
data_train[binary_cols].head()

Binary columns after encoding (sample):


,potential_issue,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,0,0,0,0,1,0,0
1,0,0,0,0,1,0,0
2,0,1,0,0,1,0,0
3,0,0,0,0,1,0,0
4,0,1,0,0,1,0,0


In [38]:
print("\nPerformance categories after one-hot encoding (sample):")
perf_dummy_cols = [col for col in data_train.columns if col.startswith('perf_')]
data_train[perf_dummy_cols].head()


Performance categories after one-hot encoding (sample):


,perf_6_month_cat_Above Average,perf_6_month_cat_Average,perf_6_month_cat_Below Average,perf_6_month_cat_No Data,perf_6_month_cat_Perfect,perf_12_month_cat_Above Average,perf_12_month_cat_Average,perf_12_month_cat_Below Average,perf_12_month_cat_No Data,perf_12_month_cat_Perfect
0,False,False,False,True,False,False,False,False,True,False
1,False,False,True,False,False,False,False,True,False,False
2,False,False,False,True,False,False,False,False,True,False
3,True,False,False,False,False,True,False,False,False,False
4,False,False,False,True,False,False,False,False,True,False


## Feature Scaling
Finding: Numerical features operate on vastly different scales (e.g., inventory vs. lead time). 

Action: Apply scaling to numerical features (after transformations like log).

we use StandardScaler because:
- data is already log-transformed
- features centered around 0 with unit variance
- works well with most ML algorithms

In [39]:
from sklearn.preprocessing import StandardScaler

# Identify numerical columns to scale (excluding binary and one-hot encoded columns)
numerical_cols = ['national_inv', 'lead_time', 'in_transit_qty',
                  'forecast_3_month', 'forecast_6_month', 'forecast_9_month',
                  'sales_1_month', 'sales_3_month', 'sales_6_month', 
                  'sales_9_month', 'min_bank', 'pieces_past_due',
                  'local_bo_qty']

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on training data only
scaler.fit(data_train[numerical_cols])

# Transform both training and test data
data_train[numerical_cols] = scaler.transform(data_train[numerical_cols])
data_test[numerical_cols] = scaler.transform(data_test[numerical_cols])

# Verify the scaling worked as expected
print("Numerical features after scaling (train data):")
print(data_train[numerical_cols].describe().round(2))

# Verify that the scaled features have mean ≈ 0 and std ≈ 1
means = data_train[numerical_cols].mean()
stds = data_train[numerical_cols].std()

print("\nVerification - features should have mean ≈ 0 and std ≈ 1:")
print("\nMeans:")
print(means.round(4))
print("\nStandard deviations:")
print(stds.round(4))

Numerical features after scaling (train data):
       national_inv   lead_time  in_transit_qty  forecast_3_month  \
count    1687860.00  1687860.00      1687860.00        1687860.00   
mean          -0.00       -0.00           -0.00              0.00   
std            1.00        1.00            1.00              1.00   
min           -1.56       -1.15           -0.42             -0.55   
25%           -0.76       -0.57           -0.42             -0.55   
50%           -0.18        0.02           -0.42             -0.55   
75%            0.63        0.02           -0.42              0.25   
max            6.59        6.45            8.49              6.50   

       forecast_6_month  forecast_9_month  sales_1_month  sales_3_month  \
count        1687860.00        1687860.00     1687860.00     1687860.00   
mean               0.00             -0.00           0.00           0.00   
std                1.00              1.00           1.00           1.00   
min               -0.62        

## Save Processed Data

In [43]:
# First, let's see what features we have
print("Final features:", data_train.columns.tolist())
print("\nTotal number of features:", len(data_train.columns))

Final features: ['sku', 'national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month', 'forecast_6_month', 'forecast_9_month', 'sales_1_month', 'sales_3_month', 'sales_6_month', 'sales_9_month', 'min_bank', 'potential_issue', 'pieces_past_due', 'local_bo_qty', 'deck_risk', 'oe_constraint', 'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder', 'perf_6_month_cat_Above Average', 'perf_6_month_cat_Average', 'perf_6_month_cat_Below Average', 'perf_6_month_cat_No Data', 'perf_6_month_cat_Perfect', 'perf_12_month_cat_Above Average', 'perf_12_month_cat_Average', 'perf_12_month_cat_Below Average', 'perf_12_month_cat_No Data', 'perf_12_month_cat_Perfect']

Total number of features: 31


In [44]:
# Save to CSV
data_train.to_csv('../data/train_processed.csv', index=False)
data_test.to_csv('../data/test_processed.csv', index=False)

print("\nDatasets saved successfully!")
print(f"Train shape: {data_train.shape}")
print(f"Test shape: {data_test.shape}")


Datasets saved successfully!
Train shape: (1687860, 31)
Test shape: (242075, 31)


## Summary of Feature Engineering

1. **Handled Missing & Invalid Data**
   - Dropped incomplete rows in 'went_on_backorder'
   - Filled missing lead_time with median
   - Clipped negative inventory values to 0

2. **Log Transformation**
   - Applied to highly skewed numerical features
   - Used np.log1p to handle zeros
   - Significantly reduced skewness

3. **Categorical Encoding**
   - Converted binary Yes/No columns to 0/1
   - Created categorical versions of performance metrics
   - Applied one-hot encoding to performance categories

4. **Feature Scaling**
   - Applied StandardScaler to numerical features
   - Fit on training data only
   - Applied same transformation to test data

Next Steps (for modeling notebook):
- Feature selection
- Feature importance analysis
- Model-specific feature engineering if needed